# 크롤링 데이터(리뷰) 전처리, 레이블링

In [60]:
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
#from pykospacing import Spacing
from collections import Counter

In [61]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

In [62]:
import json

with open('./naver_횟집_by_metropolitan2.json','r',encoding='utf-8') as f:
    data = json.load(f)

In [63]:
#매장의 개수
len(data)

1775

## 전처리

1. 각 매장 마다의 리뷰 리스트를 나누고
2. null값 확인
3. 한글빼고 삭제 -> pass
4. 중복제거
5. 길이 10이하 텍스트 제거
5. 레이블링

In [64]:
df = pd.DataFrame.from_dict(data, orient='index')
df.reset_index(drop=False, inplace=True)
df

,index,location,review,review_rank,rank
0,청초수물회 송파점,서울 송파구 백제고분로21길 38 1층 102호,[],[],NaN
1,두껍다회선생,서울 성북구 성북로 24 1층,[평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음....,"[5, 4, 4.5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4.5,...",4.41
2,목포회센타,서울 관악구 남부순환로 1766,[오늘은 광어가 좀 비리네요. 사장님께 말했더니 죄송\n하다고 담엔 바로 얘기하라고...,"[5, 4, 4.5, 4.5, 1, 5, 5, 0.5, 0.5, 4, 4, 4.5,...",NaN
3,바다회사랑,대전 중구 계룡로874번길 89,"[저렴한가격에 양 많아요, 좋아요, 굿]","[3.5, 4, 5]",3.61
4,은성횟집,서울 종로구 창경궁로 88,"[광장시장 근본 대구탕,,,\n조금 시끄럽긴 해도 맛은 굳,,,, 약간 짠편이라 안...","[5, 5, 1, 4, 4.5, 3.5, 3, 1, 5, 5, 4, 3, 4, 3,...",4.27
...,...,...,...,...,...
1770,청연 덕신점,울산 울주군 온산읍 온덕2길 13 1층 청연 덕신점,[],[],4.74
1771,회명품,울산 동구 남진길 63,[],[],NaN
1772,다해횟집,울산 북구 정자1길 23-1 다해횟집,"[사장님 친절하시고 음식도맛있고 좋아요~\n서비스도 엄청 좋아요~, 깨끗하고 친절해...","[5, 3.5, 3, 4]",4.44
1773,더참치,울산 북구 동대중앙로 27 1층 더 참치,"[사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥...","[3, 0.5, 4]",3.43


In [65]:
df.rename(columns={'index': 'store_name'}, inplace=True)
df

,store_name,location,review,review_rank,rank
0,청초수물회 송파점,서울 송파구 백제고분로21길 38 1층 102호,[],[],NaN
1,두껍다회선생,서울 성북구 성북로 24 1층,[평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음....,"[5, 4, 4.5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4.5,...",4.41
2,목포회센타,서울 관악구 남부순환로 1766,[오늘은 광어가 좀 비리네요. 사장님께 말했더니 죄송\n하다고 담엔 바로 얘기하라고...,"[5, 4, 4.5, 4.5, 1, 5, 5, 0.5, 0.5, 4, 4, 4.5,...",NaN
3,바다회사랑,대전 중구 계룡로874번길 89,"[저렴한가격에 양 많아요, 좋아요, 굿]","[3.5, 4, 5]",3.61
4,은성횟집,서울 종로구 창경궁로 88,"[광장시장 근본 대구탕,,,\n조금 시끄럽긴 해도 맛은 굳,,,, 약간 짠편이라 안...","[5, 5, 1, 4, 4.5, 3.5, 3, 1, 5, 5, 4, 3, 4, 3,...",4.27
...,...,...,...,...,...
1770,청연 덕신점,울산 울주군 온산읍 온덕2길 13 1층 청연 덕신점,[],[],4.74
1771,회명품,울산 동구 남진길 63,[],[],NaN
1772,다해횟집,울산 북구 정자1길 23-1 다해횟집,"[사장님 친절하시고 음식도맛있고 좋아요~\n서비스도 엄청 좋아요~, 깨끗하고 친절해...","[5, 3.5, 3, 4]",4.44
1773,더참치,울산 북구 동대중앙로 27 1층 더 참치,"[사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥...","[3, 0.5, 4]",3.43


In [66]:
df = df.drop(['location', 'rank'],axis=1)
df

,store_name,review,review_rank
0,청초수물회 송파점,[],[]
1,두껍다회선생,[평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음....,"[5, 4, 4.5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4.5,..."
2,목포회센타,[오늘은 광어가 좀 비리네요. 사장님께 말했더니 죄송\n하다고 담엔 바로 얘기하라고...,"[5, 4, 4.5, 4.5, 1, 5, 5, 0.5, 0.5, 4, 4, 4.5,..."
3,바다회사랑,"[저렴한가격에 양 많아요, 좋아요, 굿]","[3.5, 4, 5]"
4,은성횟집,"[광장시장 근본 대구탕,,,\n조금 시끄럽긴 해도 맛은 굳,,,, 약간 짠편이라 안...","[5, 5, 1, 4, 4.5, 3.5, 3, 1, 5, 5, 4, 3, 4, 3,..."
...,...,...,...
1770,청연 덕신점,[],[]
1771,회명품,[],[]
1772,다해횟집,"[사장님 친절하시고 음식도맛있고 좋아요~\n서비스도 엄청 좋아요~, 깨끗하고 친절해...","[5, 3.5, 3, 4]"
1773,더참치,"[사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥...","[3, 0.5, 4]"


In [67]:
print(sum(df['review_rank'].apply(len) == 0))
zero_review_index = df[df['review'].apply(len) == 0].index
df = df.drop(zero_review_index)

121


In [70]:
df = df.reset_index(drop=True)

In [71]:
df

,store_name,review,review_rank
0,두껍다회선생,[평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음....,"[5, 4, 4.5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4.5,..."
1,목포회센타,[오늘은 광어가 좀 비리네요. 사장님께 말했더니 죄송\n하다고 담엔 바로 얘기하라고...,"[5, 4, 4.5, 4.5, 1, 5, 5, 0.5, 0.5, 4, 4, 4.5,..."
2,바다회사랑,"[저렴한가격에 양 많아요, 좋아요, 굿]","[3.5, 4, 5]"
3,은성횟집,"[광장시장 근본 대구탕,,,\n조금 시끄럽긴 해도 맛은 굳,,,, 약간 짠편이라 안...","[5, 5, 1, 4, 4.5, 3.5, 3, 1, 5, 5, 4, 3, 4, 3,..."
4,횟집울릉도,[#맛있음 #그러나 #계산법이 #괴상함 모듬물회 5인분 포장주문하려하니 포장용기가 ...,"[1, 2, 2, 4, 5, 5, 4, 5, 4.5, 0.5, 5, 3.5, 1, ..."
...,...,...,...
1649,횟집하늘바다,[매번 갈때마다 단골마다 서비스도 거하게 주시고 잘\n먹었습니다 서울에서 오신 손님...,"[5, 5]"
1650,어람횟집,"[맛있어요~^^, ㅎㅎ, 좋아요]","[4, 4, 5]"
1651,다해횟집,"[사장님 친절하시고 음식도맛있고 좋아요~\n서비스도 엄청 좋아요~, 깨끗하고 친절해...","[5, 3.5, 3, 4]"
1652,더참치,"[사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥...","[3, 0.5, 4]"


In [72]:
df.isnull().sum()
sum(df['review_rank'].apply(len) == 0)

0

In [73]:
#1. 각 매장 마다의 리뷰 리스트를 하나의 리뷰 코퍼스로 만든다
review_list = list()
review_rank_list = list()
for store_idx in df.index:
    review_list.extend(df['review'][store_idx])
    review_rank_list.extend(df['review_rank'][store_idx])
print(review_list[:5])
print(review_rank_list[:5])
    

['평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. 두꺼운 회의 매력에 빠졌달까 ㅎㅎ 우선 생와사비 주는 거 넘나 좋...', '숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가 싸서 이해는하지만\n ㅠ 매운탕 가성비있어서 조아요오 :)', '남자 사장님친절하세요\n여자분은별로...', '근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니 좋네요', '맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용']
['5', '4', '4.5', '4', '5']


In [74]:
print(len(review_list))
len(review_rank_list)

87549


87549

In [75]:
each_review_index = list()
for df_idx in range(len(df)):
    for i in range(len(df.loc[df_idx,'review'])): #각 매장의 리뷰 리스트 개수만큼
        each_review_index.append(df.loc[df_idx,'store_name'])
len(each_review_index)

87549

In [76]:
#데이터 프레임 생성
df = pd.DataFrame({'Store_name' : each_review_index, 'Review': review_list, 'Rank': review_rank_list})
df

,Store_name,Review,Rank
0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. ...,5
1,두껍다회선생,숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가...,4
2,두껍다회선생,남자 사장님친절하세요\n여자분은별로...,4.5
3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,두껍다회선생,맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...
87544,더참치,참치 맛이 조~~~금 비리대요,4
87545,일품,물회맛있어요~\n매운탕도 같이줘서 굿굿,4.5
87546,일품,맛있고 좋아요,4.5
87547,일품,우럭탕 맛있어여~~^^,5


In [77]:
# null값 확인
df.isnull().sum()

Store_name       0
Review        1820
Rank             0
dtype: int64

In [83]:
# 평점은 있지만 리뷰가 없는 데이터 삭제
df[df['Review'].isnull()]
df = df.dropna()
df = df.reset_index()

In [84]:
df

,index,Store_name,Review,Rank
0,0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. ...,5
1,1,두껍다회선생,숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가...,4
2,2,두껍다회선생,남자 사장님친절하세요\n여자분은별로...,4.5
3,3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,4,두껍다회선생,맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...,...
85724,87544,더참치,참치 맛이 조~~~금 비리대요,4
85725,87545,일품,물회맛있어요~\n매운탕도 같이줘서 굿굿,4.5
85726,87546,일품,맛있고 좋아요,4.5
85727,87547,일품,우럭탕 맛있어여~~^^,5


In [85]:
# 한글이 아닌 문자 제거 -> pass
#여러 공백을 하나의 공백으로 줄인다.
import re
def apply_regular_expreesion(text):
    # 특수문자, 숫자, 구두점 제거
    comp = re.compile('[^ a-zA-Z가-힣]')
    result = comp.sub('', text)
    #여러 공백 하나의 공백으로 변경 
    result = re.sub(' +',' ', text)
    return result

df['Review'] = df['Review'].apply(lambda x : apply_regular_expreesion(x))

In [86]:
tmp = df[df['Review']=='']
tmp

,index,Store_name,Review,Rank
117,118,목포회센타,,4
1953,2011,형제상회,,4.5
1963,2021,형제상회,,4
1966,2024,형제상회,,5
2727,2796,고기야미안해,,5
...,...,...,...,...
82586,84252,섬뜰,,5
83105,84790,소쿠리횟집,,4
83422,85111,두리두리 병영점,,5
84899,86666,직송회직판장,,5


In [89]:
sum(df['Review']==' ')


0

In [90]:
#빈 문자열 -> NaN 치환
df['Review'].replace('', np.nan, inplace=True)
# null값 확인
print(df.isnull().sum())
df.dropna(inplace=True)
len(df)

index          0
Store_name     0
Review        83
Rank           0
dtype: int64


85646

In [91]:
df.duplicated(['Review']).sum()

23257

In [92]:
#중복 제거
df = df.drop_duplicates(['Review'])
print(df.duplicated(['Review']).sum())
print(len(df))

0
62389


In [94]:
print(df.isnull().sum())

index         0
Store_name    0
Review        0
Rank          0
dtype: int64


In [95]:
len(df)

62389

In [97]:
df = df.reset_index(drop=True)

In [98]:
df

,index,Store_name,Review,Rank
0,0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. ...,5
1,1,두껍다회선생,숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가...,4
2,2,두껍다회선생,남자 사장님친절하세요\n여자분은별로...,4.5
3,3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,4,두껍다회선생,맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...,...
62384,87542,더참치,"사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥저...",3
62385,87543,더참치,엄마 생신이라 평소 가던곳 아닌 더참치 처음 갔어요 . 41만원나왔고. 1인 열점먹...,0.5
62386,87544,더참치,참치 맛이 조~~~금 비리대요,4
62387,87545,일품,물회맛있어요~\n매운탕도 같이줘서 굿굿,4.5


In [99]:
import pickle
with open('df.pickle' , 'wb') as f:
    pickle.dump(df,f)

In [100]:
df

,index,Store_name,Review,Rank
0,0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. ...,5
1,1,두껍다회선생,숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가...,4
2,2,두껍다회선생,남자 사장님친절하세요\n여자분은별로...,4.5
3,3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,4,두껍다회선생,맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...,...
62384,87542,더참치,"사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥저...",3
62385,87543,더참치,엄마 생신이라 평소 가던곳 아닌 더참치 처음 갔어요 . 41만원나왔고. 1인 열점먹...,0.5
62386,87544,더참치,참치 맛이 조~~~금 비리대요,4
62387,87545,일품,물회맛있어요~\n매운탕도 같이줘서 굿굿,4.5


In [102]:
df = df.astype({'Rank':'float'})

In [103]:
df.dtypes

index           int64
Store_name     object
Review         object
Rank          float64
dtype: object

In [109]:
#조건별 데이터 연산
#3~5점 긍정
#0~1점 부정
# 레이블링
df['Label'] = np.select([df.Rank > 3], [1], default= 0)
df[:5]

,index,Store_name,Review,Rank,Label
0,0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. ...,5.0,1
1,1,두껍다회선생,숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가...,4.0,1
2,2,두껍다회선생,남자 사장님친절하세요\n여자분은별로...,4.5,1
3,3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4.0,1
4,4,두껍다회선생,맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용,5.0,1


In [110]:
import pickle
with open('df.pickle' , 'wb') as f:
    pickle.dump(df,f)

# 2. 레이블링
1. MeCab으로 형태소 분석
2. 리뷰의 모든 단어 사전 생성
3. 단어 사전에서 긍정지정사(VCP), 부정지정사(VCN), 부사(MAG), 형용사(VA), 동사(VV), 감탄사(IC), 어근(XR), 명사(NNG), 그리고 이 형태소가 포함되어 있는 태그들을 감성어 사전에 포함 => 판단의 근거가 된다.
4. 긍정어 사전에 있던 형태소들에는 1을
5. 부정어 사전에 있던 형태소들에는 -1을 태깅

In [86]:
# 형태소 분석기 로드
from konlpy.tag import Mecab
mecab = Mecab('C:\mecab\mecab-ko-dic')

In [87]:
mecab.pos('안녕하세요. 저는 정성원입니다.')

[('안녕', 'NNG'),
 ('하', 'XSV'),
 ('세요', 'EP+EF'),
 ('.', 'SF'),
 ('저', 'NP'),
 ('는', 'JX'),
 ('정성원', 'NNP'),
 ('입니다', 'VCP+EF'),
 ('.', 'SF')]

In [88]:
store_name_col = pd.DataFrame(df['Store_name'])
store_name_col

,Store_name
0,강세호참치
1,강세호참치
2,강세호참치
3,강세호참치
4,강세호참치
...,...
86641,팔딱팔딱
86642,팔딱팔딱
86643,팔딱팔딱
86644,팔딱팔딱


In [89]:
# 전체 단어 사전 생성
pos_tag_corpus_list = []
for index, row in tqdm(df.iterrows(), total = len(df)):
     pos_tag_corpus_list.append(mecab.pos(row['Review']))

100%|█████████████████████████████████████████████████████████████████| 86646/86646 [00:08<00:00, 10607.95it/s]


In [90]:
pos_tag_corpus_list[0]

[('지금', 'NNG'),
 ('까지', 'JX'),
 ('다녀', 'VV+EC'),
 ('봤', 'VX+EP'),
 ('던', 'ETM'),
 ('참치', 'NNG'),
 ('집', 'NNG'),
 ('중', 'NNB'),
 ('부위', 'NNG'),
 ('가', 'JKS'),
 ('가장', 'MAG'),
 ('좋', 'VA'),
 ('았', 'EP'),
 ('던', 'ETM'),
 ('거', 'NNB'),
 ('같', 'VA'),
 ('아요', 'EF'),
 ('!', 'SF'),
 ('어묵', 'NNG'),
 ('탕', 'NNG'),
 ('샐러드', 'NNG'),
 ('생선', 'NNG'),
 ('조림', 'NNG'),
 ('순두부찌개', 'NNG'),
 ('튀김', 'NNG'),
 ('등', 'NNB'),
 ('곁들일', 'VV+ETM'),
 ('수', 'NNB'),
 ('있', 'VV'),
 ('는', 'ETM'),
 ('스', 'IC'),
 ('끼', 'MAG'),
 ('다시', 'MAG'),
 ('도', 'JX'),
 ('같이', 'MAG'),
 ('나와서', 'VV+EC'),
 ('좋', 'VA'),
 ('아', 'EF'),
 ('.', 'SF'),
 ('..', 'SY')]

In [91]:
#3. 단어 사전에서 긍정지정사(VCP), 부정지정사(VCN), 부사(MAG), 형용사(VA), 동사(VV), 감탄사(IC), 어근(XR), 그리고 이 형태소가 포함되어 있는 태그들을 감성어 사전에 포함
import re
filter_poses = ['VCP', 'VCN','MAG','VA','VV','IC','XR']

filtered_morphs_corpus_list = []
review_Lex = []

for tokenized_review in tqdm(pos_tag_corpus_list, total=len(pos_tag_corpus_list)):# 한줄의 리뷰
    tmp = []
    for morphs_pos_tuple in tokenized_review: # 한 형태소_품사 tuple
        for test_pos in filter_poses: #한 형태소
            m = re.search(test_pos, morphs_pos_tuple[1]) # 필터 품사와 현재 형태소 품사 비교
            if m != None: #그 형태소의 품사가 품사 필터를 포함하고 있으면 break
                break
        if m == None: #아직도 None이라면 이 형태소는 단어사전에 포함시키지 않는다.
            continue
        tmp.append(morphs_pos_tuple[0])
        review_Lex.append(morphs_pos_tuple[0])
    filtered_morphs_corpus_list.append(tmp)
    

100%|█████████████████████████████████████████████████████████████████| 86646/86646 [00:07<00:00, 11097.10it/s]


In [93]:
len(df), len(filtered_morphs_corpus_list)

(86646, 86646)

In [94]:
filtered_morphs_corpus_list[0]

['다녀', '가장', '좋', '같', '곁들일', '있', '스', '끼', '다시', '같이', '나와서', '좋']

In [95]:
df

,Store_name,Review
0,강세호참치,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요! 어묵탕 샐러드 생선조림 ...
1,강세호참치,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나? 벨을 눌러도 안오네요ㅋ...
2,강세호참치,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...
3,강세호참치,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...
4,강세호참치,음식이 깔끔하고 맛있네요
...,...,...
86641,팔딱팔딱,동네에서 회가 맛있다고 소문 듣고 방문했는데 정말 \n회가 신선하고 맛있어요😋 가성...
86642,팔딱팔딱,맛있어서 이번엔 모임있어서 포장해갔어요\n윤기 좔좔✨️✨️✨️\n회 좋아하시는분들은...
86643,팔딱팔딱,우러이 쫄깃쫄깃합니다
86644,팔딱팔딱,신선하고 그외 반찬 나오는것도 맛있고\n매운탕도 굿~


In [96]:
review_Lex[:10]

['다녀', '가장', '좋', '같', '곁들일', '있', '스', '끼', '다시', '같이']

In [98]:
#각 리뷰별 필터링된 토큰들
for idx in range(len(df)):
    df.at[idx,'Tokenized_review'] = np.array(filtered_pos_tag_corpus_list[idx]).astype(object)

In [99]:
df.head()

,Store_name,Review,Tokenized_review
0,강세호참치,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요! 어묵탕 샐러드 생선조림 ...,"[다녀, 가장, 좋, 같, 곁들일, 있, 스, 끼, 다시, 같이, 나와서, 좋]"
1,강세호참치,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나? 벨을 눌러도 안오네요ㅋ...,"[처음, 들어갈, 계신, 같, 먹, 눌러, 안, 오, ㅋㅋ, 보, 이, 노래]"
2,강세호참치,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...,"[다, 좋아해서, 맛있, 회, 먹, 찾아보, 알, 된, 이, 이]"
3,강세호참치,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...,"[계속, 다닐, 에요, 첨, 뵙, 우린, 많이, 준]"
4,강세호참치,음식이 깔끔하고 맛있네요,"[깔끔, 맛있]"


In [100]:
#전체 사전
review_Lex[:10]

['다녀', '가장', '좋', '같', '곁들일', '있', '스', '끼', '다시', '같이']

In [101]:
#전체 사전 중복 제거
review_Lex = list(set(review_Lex))
len(review_Lex)

7044

In [93]:
with open('review_Lex.pickle', 'wb') as f:
    pickle.dump(review_Lex, f)
with open('df.pickle', 'wb') as f:
    pickle.dump(df, f)
with open('filtered_pos_tag_corpus_list','wb') as f:
    pickle.dump(filtered_pos_tag_corpus_list,f)

In [15]:
with open('review_Lex.pickle', 'rb') as f:
    review_Lex = pickle.load(f)
with open('df.pickle', 'rb') as f:
    df = pickle.load(f)
with open('filtered_pos_tag_corpus_list','rb') as f:
    filtered_pos_tag_corpus_list = pickle.load(f)

In [102]:
review_Lex[:5]

['아닌가', '이런가', '준다며', '대체', '탕탕']

# 감성사전 생성
## 기존 KNU단어 사전 이용
## KNU 긍정/ 부정 사전에 없으면 직접 추가

In [104]:
#각 단어에 긍정 / 부정 점수 부여
SentiWord = dict()
for i in range(100):
    print(f'{i}번째 형태소')
    target_morphs = review_Lex[i]
    print(target_morphs)
    label = input("레이블 입력하세요: ")
    label = int(label)
    SentiWord[target_morphs] = label
    

0번째 형태소
아닌가
레이블 입력하세요: -1
1번째 형태소
이런가
레이블 입력하세요: 1
2번째 형태소
준다며
레이블 입력하세요: 1
3번째 형태소
대체
레이블 입력하세요: 1
4번째 형태소
탕탕
레이블 입력하세요: 1
5번째 형태소
묻혀져
레이블 입력하세요: 1
6번째 형태소
댁
레이블 입력하세요: 1
7번째 형태소
떠오
레이블 입력하세요: 1
8번째 형태소
왁
레이블 입력하세요: 1
9번째 형태소
무지막지
레이블 입력하세요: 1
10번째 형태소
씌운
레이블 입력하세요: 1
11번째 형태소
부러워
레이블 입력하세요: 1
12번째 형태소
챙겨서
레이블 입력하세요: 1
13번째 형태소
기러


KeyboardInterrupt: Interrupted by user

In [10]:
#현재 인덱스 2500까지 했다.
for i in range(df['Label'].count(), 2500):
    print(f'{i}번째 인덱스')
    print(df.loc[i,'Review'])
    label = input("레이블 입력하세요: ")
    label = int(label)
    df.loc[i,'Label'] = label

2993번째 인덱스
나오는 기타 요리들도 너무 맛있고 무엇보다 대게가 좋습니다 찌시는 방법도 직접 솥에 쪄주셔서 정성이 들어가 더 맛있는 것 같아요 너무 친절하셔서 
레이블 입력하세요: 1
2994번째 인덱스
너무 맛있게 잘 먹었습니다 사장님도 친절하시고 무엇보다 킹크랩이 엄청 살도 꽉차고 진짜 진짜 맛있습니다 무조건 재방문각입니다
레이블 입력하세요: 1
2995번째 인덱스
동교동 창천동 모두 먹었던 곳에서 제일 만족한 곳이에요 서비스는 물론이고 기본 채소 퀄리티부터가 정말 진짜 추천 많이합니다 기분좋아지는 것
레이블 입력하세요: 1
2996번째 인덱스
제가 사진찍는 기술이 후져서 실제 나온 것보다 못찍어서 아쉽네요 잘먹었습니다
레이블 입력하세요: 1
2997번째 인덱스
대게맛의 진수 서비스도 많고 정말 맛있습니다
레이블 입력하세요: 1
2998번째 인덱스
먹기 좋게 손쉽게 손질되어 나오고 속살도 꽉 차서 진짜 맛있게 잘 먹었습니다신선해서 그런지 비린내도 전혀 안났어요
레이블 입력하세요: 1
2999번째 인덱스
참 맛있게 먹고 갑니다
레이블 입력하세요: 1


In [10]:
# df_tmp 저장
import pickle
with open('df_tmp.pickle', 'rb') as f:
    df = pickle.load(f)

In [11]:
df

,Review,Label
0,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요 어묵탕 샐러드 생선조림 순...,1.0
1,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나 벨을 눌러도 안오네요 보...,0.0
2,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...,1.0
3,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...,1.0
4,음식이 깔끔하고 맛있네요,1.0
...,...,...
83564,방어 넘 양도 많고 짱이네요포장도 갠찬아요,NaN
83565,횟감이 신선하고 쫀득한,NaN
83566,사장님 친절하시고 음식도맛있고 좋아요서비스도 엄청 좋아요,NaN
83567,깨끗하고 친절해요 맘에듭니다,NaN


In [16]:
import pandas as pd
import numpy as np
df.value_counts(['Label']).sum()

2500

In [32]:
# 5000개의 데이터 랜덤 추출
tmp = df.sample(n=5000, replace = False, random_state=24)

In [33]:
tmp.count()

Review    5000
Label      150
dtype: int64

In [38]:
# 기존의 순차적 레이블링 진행했던 2500개의 레코드 df
# 새롭게 비복원 추출로 랜덤하게 뽑은 5000개 tmp를 합친다.
df = pd.concat([df.iloc[:2500,:], tmp], ignore_index=True)

In [39]:
df.count()

Review    7500
Label     2650
dtype: int64

In [41]:
# 중복제거
df.drop_duplicates(['Review'],ignore_index=True, inplace=True)
df.count()

Review    7350
Label     2500
dtype: int64

In [79]:
for i in range(df['Label'].count(), 3000):
    print(f'{i}번째 인덱스')
    print(df.loc[i,'Review'])
    label = input("레이블 입력하세요: ")
    label = int(label)
    df.loc[i,'Label'] = label

2940번째 인덱스
반찬들 하나하나 다 잘나왔어요 청어구이보다는 꽁 치가 더 좋을것 같긴 한데 방어 먹으러 왔는데 메뉴에 없는 방어참가자미 반반으로 해주셔
레이블 입력하세요: 1
2941번째 인덱스
방어우럭 포장했는데 광어우럭이 있네요 바쁠때 아니었는데 포장하기전에 확인해주시면 좋겠어요
레이블 입력하세요: 1
2942번째 인덱스
회가 너무 신선하고 맛있었어요
레이블 입력하세요: 1
2943번째 인덱스
혼밥 해서 죄송하지만 청결하고 친절하고음식도 신선하고 너무 좋아요 또갈게요
레이블 입력하세요: 1
2944번째 인덱스
위생 상태도 너무 좋고 맛있게 잘 먹었어요 그날 하 루 배가 터질 지경이었어요 상다리가 부러지게 나와서 회 상태도 너무 신선하고 좋더라
레이블 입력하세요: 1
2945번째 인덱스
항상친절하고 좋아요 노량진 갈때마다 무조건 청해진
레이블 입력하세요: 1
2946번째 인덱스
모듬회하고 물회 너무 맛있게 먹었어요초밥도 팔던데 다음에 재방문해서 먹어보려구요
레이블 입력하세요: 1
2947번째 인덱스
너무 맛있게 먹고 갓네요 서비스나 회 신선도나친절함이나 최고에요
레이블 입력하세요: 1
2948번째 인덱스
룸으로 배정 받아서 더 좋은시간 보냈네요
레이블 입력하세요: 1
2949번째 인덱스
맛있어요 점심특선 라멘은 좀 그랬어요
레이블 입력하세요: 1
2950번째 인덱스
차 차로 오는 사람들이 많다보니 많이 시끄럽고 정신이 없어요 
레이블 입력하세요: 0
2951번째 인덱스
자주가는 참치집입니다 맛 친절 양 질 다 좋아용
레이블 입력하세요: 1
2952번째 인덱스
최고였네요 싱싱하고 가성비 좋았습니다 기분 좋게 먹고 나왔어요 친구들에게도 추천하고 싶은 집입니다
레이블 입력하세요: 1
2953번째 인덱스
광어포차 좋아요친절하고 맛있어요
레이블 입력하세요: 1
2954번째 인덱스
회랑 새우튀김도 매운탕도 맛있었어요
레이블 입력하세요: 1
2955번째 인덱스
맛있고 가격도 좋아요
레이블 입력하세요: 1
2956번째 인덱스
여러가지 종류의 회와 해산물이 들어있어서 여

In [84]:
df.value_counts(['Label']).sum()

3000

In [87]:
result = df.iloc[:3000,:]
result

,Review,Label
0,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요 어묵탕 샐러드 생선조림 순...,1.0
1,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나 벨을 눌러도 안오네요 보...,0.0
2,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...,1.0
3,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...,1.0
4,음식이 깔끔하고 맛있네요,1.0
...,...,...
2995,사시미 종류도 많고 양도 좋아용,1.0
2996,절대 가지마세요 점도 못줌 성남동 관평동 용문동 에서 자주 애용하면서 체인점이라도 ...,0.0
2997,양도 엄청 많고 맛잇어용,1.0
2998,너무 맛있어요 사장님도 친절하셔서 기분 좋게 먹다갑니다,1.0


In [93]:
result = result.astype({'Label':'int'})

In [94]:
result

,Review,Label
0,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요 어묵탕 샐러드 생선조림 순...,1
1,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나 벨을 눌러도 안오네요 보...,0
2,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...,1
3,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...,1
4,음식이 깔끔하고 맛있네요,1
...,...,...
2995,사시미 종류도 많고 양도 좋아용,1
2996,절대 가지마세요 점도 못줌 성남동 관평동 용문동 에서 자주 애용하면서 체인점이라도 ...,0
2997,양도 엄청 많고 맛잇어용,1
2998,너무 맛있어요 사장님도 친절하셔서 기분 좋게 먹다갑니다,1


In [98]:
result['Label'].values

array([1, 0, 1, ..., 1, 1, 1])

In [101]:
# 관계없는 리뷰 삭제
drop_index = result[result['Label'] > 1].index
result.drop(drop_index,inplace=True)

In [102]:
result.reset_index(drop=True,inplace=True)

In [103]:
result.value_counts(['Label'])

Label
1        2405
0         527
dtype: int64

In [105]:
#객체 저장
import pickle
with open('result.pickle', 'wb') as f:
    pickle.dump(result,f)